In [ ]:
dbutils.widgets.text("process_id", "-1", "process_id")

In [ ]:
process_id = int(dbutils.widgets.get("process_id"))
process_id

71686511752993

In [ ]:
from etl.utils import get_environment
import logging

In [ ]:
logger = logging.getLogger(f"initialise ad_works")
env = get_environment(spark = spark)
catalog = f"{env}_hub"
spark.sql(f"USE CATALOG {catalog}")
logger.info(f"default catalog set to {catalog}")

INFO : 2023-12-04 19:20:27,719 : fnz_pb.utils : utils.py.get_environment: line(39) : Detected the dev environment
INFO : 2023-12-04 19:20:27,821 : initialise fnz_pb : command-1412197260571928-3108903530.<module>: line(5) : default catalog set to dev_hub


In [ ]:

df = spark.sql(f"""
select
  snapshot_date,
  `table`,
  file_name,
  file_size,
  schema_valid,
  total_count,
  valid_count,
  invalid_count,
  invalid_ratio,     
  process_id,
  load_date
from {catalog}._ad_works._audit
where process_id = {process_id}
order by snapshot_date, `table`
""", process_id = process_id, catalog = catalog)

display(df)

header_entity,header_api_version,fnz_header_extract_date,file_name,file_size,schema_valid,footer_reconciliation_value,footer_row_count,actual_row_count,invalid_row_count,process_id,load_date


In [ ]:
df_failed = df.where("!schema_valid")
display(df_failed)

header_entity,header_api_version,fnz_header_extract_date,file_name,file_size,schema_valid,footer_reconciliation_value,footer_row_count,actual_row_count,invalid_row_count,process_id,load_date


In [ ]:
failed_count = df_failed.count()
succeeded_count = df.count()

if failed_count > 0:
  msg = f"ingest_ad_works job to load {catalog}.ad_works load has {failed_count} file(s) with schema failures. See the finalisation step for details."
  logger.error(msg)
  raise Exception(msg)
else:
  msg = f"ingest_ad_works job to load {catalog}.ad_works load has successfully loaded {succeeded_count} files."
  dbutils.notebook.exit(msg)